In [7]:
cd /content/drive/My Drive/classifer

/content/drive/My Drive/classifer


In [8]:
# !unzip ALL.zip

In [9]:
!pip install efficientnet-pytorch==0.6.3

# train

In [10]:
import argparse
import torch
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.tensorboard import SummaryWriter

In [11]:
def load_data(input_size = 299,batch_size = 8,data_dir="None"):
    if os.path.exists(data_dir):
        print("Initializing Datasets and Dataloaders...")
    else:
        print("Error loading your data....T_T.....")
        return None
    data_transforms = {
     'train': transforms.Compose([
        transforms.Resize(input_size),
        transforms.RandomResizedCrop(input_size),
        transforms.RandomRotation(25),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        # transforms.RandomGrayscale(p=0.1),
        # torchvision.transforms.RandomApply([transforms.Lambda(lambda x: x.convert("HSV"))]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        # transforms.RandomErasing()
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.RandomResizedCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    }
    # Create training and validation datasets
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
    # Create training and validation dataloaders
    dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=8) for x in ['train', 'val']}

    return dataloaders_dict


In [12]:
def load_checkpoint(model, filename):
    print("Loading check point ..........")
    # Note: Input model & optimizer should be pre-defined.  This routine only updates their states.
    if os.path.isfile(filename):
        checkpoint = torch.load(filename)
        model.load_state_dict(checkpoint['model_state_dict'])
    else:
        print("=> no checkpoint found at '{}'".format(filename))
    return model

In [13]:
def set_parameter_requires_grad(model):
  for name, child in model.named_children():
    if name in ['Mixed_7a', 'Mixed_7b','Mixed_7c']:
      print(name + ' is unfrozen')
      for param in child.parameters():
        param.requires_grad = True
    else:
      print(name + ' is frozen')
      for param in child.parameters():
        param.requires_grad = False
  return model

In [14]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=30, path_out_module="",aux=0.5,is_inception=True):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    writer = SummaryWriter()

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + aux*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val':
                writer.add_scalar("Loss/valid_loss", epoch_loss, epoch)
                writer.add_scalar("Acc/valid_acc", epoch_acc, epoch)
            else:
                writer.add_scalar("Loss/train_loss", epoch_loss, epoch)
                writer.add_scalar("Acc/train_acc", epoch_acc, epoch)
                if epoch%20==0:
                  torch.save({
                      'model_state_dict': model.state_dict(),
                      'optimizer_state_dict': optimizer.state_dict(),
                  }, path_out_module+"_epoch_"+str(epoch)+".pt")

        
    writer.flush()
    writer.close()

In [19]:
def initialize_model(model_name="inception", num_classes=2, feature_extract = False,use_pretrained=True):
    print("Init model ....")
    model_ft = None
    if model_name == "inception":
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features   
        model_ft.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_ftrs, num_classes)
        )   
        input_size = 299
    elif model_name == "resnet":
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    elif model_name == "efficientnet-b0":
        input_size = EfficientNet.get_image_size(model_name)
        model_ft = EfficientNet.from_pretrained(model_name)

    return model_ft,input_size

In [16]:
def main(config):
    
    print("PyTorch Version: ",torch.__version__)
    print("Torchvision Version: ",torchvision.__version__)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)

    # get pretrain model && freeze some layer
    model_ft,input_size = initialize_model(config['model_name'], config['num_classes'], config['feature_extract'], use_pretrained=True)
    # loading your dataset.
    dataloaders_dict = load_data(input_size,config['batch_size'],config['path_data'])
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=1e-5, momentum=0.9,weight_decay=0.0005)
    criterion = nn.CrossEntropyLoss()
    # # 
    if config['path_continue_train_model'] is not None:
        model_ft = load_checkpoint(model_ft, config['path_continue_train_model'])
        model_ft.eval()
    
    model_ft = model_ft.to(device)
    model_ft = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, config['epochs'],config['path_out_module'],config['aux'],True)
    
    return 


In [17]:
print(torch.cuda.device_count())
config = { 
        'path_continue_train_model':None,
        'path_out_module':"ALL_MODEL_EFF/",
        'model_name':'efficientnet-b0',
        'num_classes':2,
        'path_data' :"./ALL/",
        'feature_extract':False,
        'use_pretrained':True,
        'batch_size':8,
        'input_size':299,
        'aux':0.5,
        'epochs':201
}

1


In [ ]:
main(config)

PyTorch Version:  1.5.1+cu101
Torchvision Version:  0.6.1+cu101
cuda:0
Init model ....
Loaded pretrained weights for efficientnet-b0
Initializing Datasets and Dataloaders...
Epoch 0/200
----------
